In [ ]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
"""
FVG Trading Strategy with OANDA Live Execution
Combines Fair Value Gap detection with Support/Resistance levels
"""

import pandas as pd
import numpy as np
from datetime import datetime
from apscheduler.schedulers.blocking import BlockingScheduler
from oandapyV20 import API
import oandapyV20.endpoints.orders as orders
from oandapyV20.contrib.requests import MarketOrderRequest
from oanda_candles import Pair, Gran, CandleClient
from oandapyV20.contrib.requests import TakeProfitDetails, StopLossDetails
from config import access_token, accountID
import pytz

# ============================================================================
# FVG DETECTION FUNCTIONS
# ============================================================================


def detect_fvg(data, lookback_period=14, body_multiplier=1.5):
    """
    Detects Fair Value Gaps (FVGs) in historical price data.
    """
    fvg_list = [None, None]

    for i in range(2, len(data)):
        first_high = data['High'].iloc[i-2]
        first_low = data['Low'].iloc[i-2]
        middle_open = data['Open'].iloc[i-1]
        middle_close = data['Close'].iloc[i-1]
        third_low = data['Low'].iloc[i]
        third_high = data['High'].iloc[i]

        # Calculate average body size
        prev_bodies = (data['Close'].iloc[max(0, i-1-lookback_period):i-1] -
                       data['Open'].iloc[max(0, i-1-lookback_period):i-1]).abs()
        avg_body_size = prev_bodies.mean()
        avg_body_size = avg_body_size if avg_body_size > 0 else 0.001

        middle_body = abs(middle_close - middle_open)

        # Check for Bullish FVG
        if third_low > first_high and middle_body > avg_body_size * body_multiplier:
            fvg_list.append(('bullish', first_high, third_low, i))
        # Check for Bearish FVG
        elif third_high < first_low and middle_body > avg_body_size * body_multiplier:
            fvg_list.append(('bearish', first_low, third_high, i))
        else:
            fvg_list.append(None)

    return fvg_list


def detect_key_levels(df, current_candle, backcandles=50, test_candles=10):
    """
    Detects key support and resistance levels.
    """
    key_levels = {"support": [], "resistance": []}

    last_testable_candle = current_candle - test_candles

    if last_testable_candle < backcandles + test_candles:
        return key_levels

    for i in range(current_candle - backcandles, last_testable_candle):
        high = df['High'].iloc[i]
        low = df['Low'].iloc[i]

        before = df.iloc[max(0, i - test_candles):i]
        after = df.iloc[i + 1: min(len(df), i + test_candles + 1)]

        if high > before['High'].max() and high > after['High'].max():
            key_levels["resistance"].append((i, high))

        if low < before['Low'].min() and low < after['Low'].min():
            key_levels["support"].append((i, low))

    return key_levels


def detect_break_signal(df):
    """
    Detects if current candle has FVG and previous candle crossed key level.
    Returns: 2 for bullish signal, 1 for bearish signal, 0 for no signal
    """
    df["break_signal"] = 0

    for i in range(1, len(df)):
        fvg = df.loc[i, "FVG"]
        key_levels = df.loc[i, "key_levels"]

        if isinstance(fvg, tuple) and isinstance(key_levels, dict):
            fvg_type = fvg[0]
            prev_open = df.loc[i-1, "Open"]
            prev_close = df.loc[i-1, "Close"]

            # Bullish FVG check
            if fvg_type == "bullish":
                resistance_levels = key_levels.get("resistance", [])
                for (lvl_idx, lvl_price) in resistance_levels:
                    if prev_open < lvl_price and prev_close > lvl_price:
                        df.loc[i, "break_signal"] = 2
                        break

            # Bearish FVG check
            elif fvg_type == "bearish":
                support_levels = key_levels.get("support", [])
                for (lvl_idx, lvl_price) in support_levels:
                    if prev_open > lvl_price and prev_close < lvl_price:
                        df.loc[i, "break_signal"] = 1
                        break

    return df


def process_signal_data(df, backcandles=50, test_candles=10):
    """
    Process dataframe to generate trading signals.
    """
    # Detect FVGs
    df['FVG'] = detect_fvg(df)

    # Detect key levels
    df["key_levels"] = None
    for current_candle in range(backcandles + test_candles, len(df)):
        key_levels = detect_key_levels(
            df, current_candle, backcandles, test_candles)

        support_levels = [(idx, level) for (idx, level) in key_levels["support"]
                          if idx < current_candle]
        resistance_levels = [(idx, level) for (idx, level) in key_levels["resistance"]
                             if idx < current_candle]

        if support_levels or resistance_levels:
            df.at[current_candle, "key_levels"] = {
                "support": support_levels,
                "resistance": resistance_levels
            }

    # Detect break signals
    df = detect_break_signal(df)

    return df


# ============================================================================
# OANDA TRADING FUNCTIONS
# ============================================================================

def get_candles(n, timeframe='M15'):
    """
    Fetch recent candles from OANDA.
    """
    client = CandleClient(access_token, real=False)

    # Map timeframe to Gran enum
    gran_map = {
        'M15': Gran.M15,
        'M30': Gran.M30,
        'H1': Gran.H1,
        'H4': Gran.H4,
        'D': Gran.D
    }

    collector = client.get_collector(
        Pair.EUR_USD, gran_map.get(timeframe, Gran.M15))
    candles = collector.grab(n)
    return candles


def candles_to_dataframe(candles):
    """
    Convert OANDA candles to DataFrame.
    """
    df = pd.DataFrame(columns=['Open', 'High', 'Low', 'Close'])

    for i, candle in enumerate(candles):
        df.loc[i] = {
            'Open': float(str(candle.bid.o)),
            'High': float(str(candle.bid.h)),
            'Low': float(str(candle.bid.l)),
            'Close': float(str(candle.bid.c))
        }

    df = df.astype(float)
    return df


def trading_job():
    """
    Main trading job executed on schedule.
    """
    print(f"\n{'='*60}")
    print(f"Trading Job Started: {datetime.now()}")
    print(f"{'='*60}")

    try:
        # Fetch enough candles for signal processing
        n_candles = 100  # Increased to accommodate lookback periods
        candles = get_candles(n_candles)
        df = candles_to_dataframe(candles)

        print(f"Fetched {len(df)} candles")

        # Process signals
        df = process_signal_data(df, backcandles=50, test_candles=10)

        # Get the latest signal (last row)
        signal = df['break_signal'].iloc[-1]

        print(f"Current Signal: {signal} (0=None, 1=Bearish, 2=Bullish)")

        if signal == 0:
            print("No trading signal detected.")
            return

        # Calculate SL and TP based on previous candle
        client = API(access_token)
        tp_sl_ratio = 1.8

        current_close = df['Close'].iloc[-1]
        previous_high = df['High'].iloc[-2]
        previous_low = df['Low'].iloc[-2]

        # Position sizing (adjust units as needed)
        units = 1000

        # SELL Signal (Bearish)
        if signal == 1:
            sl = previous_high
            sl_distance = sl - current_close

            if sl_distance <= 5e-4:
                print("SL distance too small, skipping trade")
                return

            tp = current_close - tp_sl_ratio * sl_distance

            print(f"SELL Order: Units={units}, SL={sl:.5f}, TP={tp:.5f}")

            mo = MarketOrderRequest(
                instrument="EUR_USD",
                units=-units,
                takeProfitOnFill=TakeProfitDetails(price=tp).data,
                stopLossOnFill=StopLossDetails(price=sl).data
            )
            r = orders.OrderCreate(accountID, data=mo.data)
            rv = client.request(r)
            print(f"Order Response: {rv}")

        # BUY Signal (Bullish)
        elif signal == 2:
            sl = previous_low
            sl_distance = current_close - sl

            if sl_distance <= 5e-4:
                print("SL distance too small, skipping trade")
                return

            tp = current_close + tp_sl_ratio * sl_distance

            print(f"BUY Order: Units={units}, SL={sl:.5f}, TP={tp:.5f}")

            mo = MarketOrderRequest(
                instrument="EUR_USD",
                units=units,
                takeProfitOnFill=TakeProfitDetails(price=tp).data,
                stopLossOnFill=StopLossDetails(price=sl).data
            )
            r = orders.OrderCreate(accountID, data=mo.data)
            rv = client.request(r)
            print(f"Order Response: {rv}")

    except Exception as e:
        print(f"Error in trading_job: {str(e)}")
        import traceback
        traceback.print_exc()


# ============================================================================
# SCHEDULER SETUP
# ============================================================================

if __name__ == "__main__":
    print("Starting FVG Trading Bot...")
    print(f"Account ID: {accountID}")
    print("Timeframe: 15-minute candles")
    print("Strategy: FVG with Support/Resistance breaks")

    # Test the trading job once before scheduling
    # Uncomment to run immediately:
    # trading_job()

    # Schedule the trading job
    scheduler = BlockingScheduler()
    scheduler.add_job(
        trading_job,
        'cron',
        day_of_week='mon-fri',
        hour='00-23',
        minute='1,16,31,46',  # Run every 15 minutes
        start_date=datetime.now(pytz.timezone('America/Chicago')),
        timezone='America/Chicago'
    )

    print("Scheduler started. Bot will run every 15 minutes.")
    scheduler.start()

Starting FVG Trading Bot...
Account ID: 101-001-37532546-001
Timeframe: 15-minute candles
Strategy: FVG with Support/Resistance breaks
Scheduler started. Bot will run every 15 minutes.

Trading Job Started: 2025-11-05 16:01:00.054471
Fetched 100 candles
Current Signal: 0 (0=None, 1=Bearish, 2=Bullish)
No trading signal detected.

Trading Job Started: 2025-11-05 16:16:00.018508
Fetched 100 candles
Current Signal: 0 (0=None, 1=Bearish, 2=Bullish)
No trading signal detected.

Trading Job Started: 2025-11-05 16:31:00.019276
Fetched 100 candles
Current Signal: 0 (0=None, 1=Bearish, 2=Bullish)
No trading signal detected.

Trading Job Started: 2025-11-05 16:46:00.045246
Fetched 100 candles
Current Signal: 0 (0=None, 1=Bearish, 2=Bullish)
No trading signal detected.

Trading Job Started: 2025-11-05 17:01:00.052829
Fetched 100 candles
Current Signal: 1 (0=None, 1=Bearish, 2=Bullish)
SELL Order: Units=1000, SL=1.14813, TP=1.14600
Order Response: {'orderCreateTransaction': {'id': '27', 'accountID

Run time of job "trading_job (trigger: cron[day_of_week='mon-fri', hour='0-23', minute='1,16,31,46'], next run at: 2025-11-05 09:46:00 CST)" was missed by 0:02:06.767493



Trading Job Started: 2025-11-05 18:46:00.021124
Fetched 100 candles
Current Signal: 0 (0=None, 1=Bearish, 2=Bullish)
No trading signal detected.

Trading Job Started: 2025-11-05 19:01:00.059081
Fetched 100 candles
Current Signal: 0 (0=None, 1=Bearish, 2=Bullish)
No trading signal detected.

Trading Job Started: 2025-11-05 19:16:00.026155
Fetched 100 candles
Current Signal: 0 (0=None, 1=Bearish, 2=Bullish)
No trading signal detected.

Trading Job Started: 2025-11-05 19:31:00.031624
Fetched 100 candles
Current Signal: 0 (0=None, 1=Bearish, 2=Bullish)
No trading signal detected.

Trading Job Started: 2025-11-05 19:46:00.043189
Fetched 100 candles
Current Signal: 0 (0=None, 1=Bearish, 2=Bullish)
No trading signal detected.

Trading Job Started: 2025-11-05 20:01:00.027051
Fetched 100 candles
Current Signal: 0 (0=None, 1=Bearish, 2=Bullish)
No trading signal detected.

Trading Job Started: 2025-11-05 20:16:00.068440
Fetched 100 candles
Current Signal: 0 (0=None, 1=Bearish, 2=Bullish)
No tra